In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def nonlinear_system_with_disturbance(x, u, disturbance):
    """Simulate a nonlinear system with disturbance.
    Args:
        x: Current state of the system (array).
        u: Control input (scalar).
        disturbance: Random disturbance affecting the system (scalar).
    Returns:
        ndarray: The derivative of the system state.
    """
    dx1 = x[1]
    dx2 = -x[0] + u + disturbance  # Add disturbance to introduce randomness
    return np.array([dx1, dx2])

In [ ]:
def sliding_surface(x, x_hat):
    """Calculate the sliding surface for control purposes.
    Args:
        x: True state of the system (array).
        x_hat: Estimated state of the system (array).
    Returns:
        ndarray: The difference (error) between true and estimated states.
    """
    return x - x_hat

def sliding_mode_control(s):
    """Compute control input using sliding mode control technique.
    Args:
        s: Sliding surface (array).
    Returns:
        ndarray: Control action based on the sliding mode control.
    """
    k = 0.05  # Gain
    return k * np.sign(s)

In [ ]:
# Simulation parameters
t_max = 50.0
dt = 0.01
time = np.arange(0, t_max, dt)
x_true = np.zeros((len(time), 2))
x_hat = np.zeros((len(time), 2))
u = 1  # Constant input
x_true[0] = np.array([1.0, 0.0])  # Initial true state
x_hat[0] = np.array([0.9, 0.0])  # Initial estimated state
disturbance = np.random.normal(0, 0.01, len(time))  # Random disturbances

# Simulation loop
for i in range(1, len(time)):
    s = sliding_surface(x_true[i-1], x_hat[i-1])
    control = sliding_mode_control(s)
    x_true[i] = x_true[i-1] + nonlinear_system_with_disturbance(x_true[i-1], u, disturbance[i-1]) * dt
    x_hat[i] = x_hat[i-1] + (nonlinear_system_with_disturbance(x_hat[i-1], u, 0.0) + control) * dt

# Plotting
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(time, x_true[:, 0], label='True State (x1)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time, x_true[:, 0], label='True State (x1)')
plt.plot(time, x_hat[:, 0], label='Estimated State (x1)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.show()
plt.close()

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(time, x_true[:, 1], label='True State (x2)')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time, x_true[:, 1], label='True State (x2)')
plt.plot(time, x_hat[:, 1], label='Estimated State (x2)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.show()
plt.close()

In [ ]:
# with different sampling rates

# Simulation parameters
t_max = 50.0
dt_data = 0.01
dt_distu = 0.02
time_data = np.arange(0, t_max, dt_data)
time_distu = np.arange(0, t_max, dt_distu)
x_true = np.zeros((len(time_data), 2))
x_hat = np.zeros((len(time_distu), 2))
u = 1  # Constant control input

# Initial states
x_true[0] = np.array([1.0, 0.0])
x_hat[0] = np.array([0.9, 0.0])
disturbance = np.random.normal(0, 0.01, len(time_data))  # Random disturbances

# Simulation for true state
for i in range(1, len(time_data)):
    x_true[i] = x_true[i-1] + nonlinear_system_with_disturbance(x_true[i-1], u, disturbance[i-1]) * dt_data

# Simulation for estimated state
for i in range(1, len(time_distu)):
    # Note: Ensure disturbance indices match the lower sampling rate
    index_true = int(i * dt_distu / dt_data)
    s = sliding_surface(x_true[index_true], x_hat[i-1])
    control = sliding_mode_control(s)
    x_hat[i] = x_hat[i-1] + (nonlinear_system_with_disturbance(x_hat[i-1], u, 0.0) + control) * dt_distu

# Plotting
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(time_data, x_true[:, 0], label='True State (x1)')
plt.xlabel('Time')
plt.ylabel('State')
plt.subplot(2, 1, 2)
plt.plot(time_data, x_true[:, 0], label='True State (x1)')
plt.plot(time_distu, x_hat[:, 0], label='Estimated State (x1)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.show()
plt.close()

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(time_data, x_true[:, 1], label='True State (x2)')
plt.subplot(2, 1, 2)
plt.plot(time_data, x_true[:, 1], label='True State (x2)')
plt.plot(time_distu, x_hat[:, 1], label='Estimated State (x2)')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.show()
plt.close()